# 编写 Leaf 系统
关于如何运行这些教程笔记本的说明，请参见 [索引](./index.ipynb)。


# 问题说明

本笔记本旨在系统性地介绍如何在 Drake 中编写自定义的 LeafSystem，主要解决以下问题：

- 如何声明和使用输入输出端口（向量值与抽象值）

- 如何声明和管理系统状态（离散、连续、抽象）

- 如何声明和访问参数

- 如何实现发布事件

- 如何为向量值命名，便于访问

- 如何支持标量类型转换（double、AutoDiff、Symbolic）

通过具体示例，帮助用户掌握自定义动力系统的完整流程。

In [1]:
import numpy as np
from pydrake.common.containers import namedview
from pydrake.common.value import Value
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import BasicVector, LeafSystem
from pydrake.trajectories import PiecewisePolynomial

## 1. 概述

[建模动力系统](./dynamical_systems.ipynb) 教程对 Drake 的系统框架进行了非常基础的介绍，包括如何编写一个基础的 [LeafSystem](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html)。在本笔记本中，我们将对如何编写这些系统进行更高级/完整的介绍。

## 2. 输入和输出端口

Leaf 系统可以拥有任意数量的输入和输出端口。端口有两种基本类型：**向量值**端口和**抽象值**端口。

### 2.1 向量值端口
我们先从最容易使用的向量值端口开始。下面的系统声明了两个向量输入端口和两个向量输出端口。它对两个 2 元输入向量分别求和与差，并将结果输出到两个输出端口。

<table align=center cellpadding=0 cellspacing=0><tr align=center style="border:none;"><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">a &rarr;</td></tr><tr style="border:none;"><td align=right style="padding:5px 0px 5px 0px; border:none;">b &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle;" bgcolor=#F0F0F0>MyAdder</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; sum</td></tr><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; difference</td></tr></table></td></tr></table>

In [ ]:
class MyAdder(LeafSystem):
    def __init__(self):
        super().__init__()  # 不要忘记初始化基类。
        self._a_port = self.DeclareVectorInputPort(name="a", size=2)
        self._b_port = self.DeclareVectorInputPort(name="b", size=2)
        self.DeclareVectorOutputPort(name="sum", size=2, calc=self.CalcSum)
        self.DeclareVectorOutputPort(name="difference",
                                     size=2,
                                     calc=self.CalcDifference)

    def CalcSum(self, context, output):
        # 通过评估输入端口获得 2x1 向量。
        a = self._a_port.Eval(context)
        b = self._b_port.Eval(context)

        # 将和写入输出向量。
        output.SetFromVector(a + b)

    def CalcDifference(self, context, output):
        # 通过评估输入端口获得 2x1 向量。
        a = self._a_port.Eval(context)
        b = self._b_port.Eval(context)

        # 将差写入输出向量。
        output.SetFromVector(a - b)

# 构造该系统的实例和一个 context。
system = MyAdder()
context = system.CreateDefaultContext()

# 将输入端口固定为常数值。
system.GetInputPort("a").FixValue(context, [3, 4])
system.GetInputPort("b").FixValue(context, [1, 2])

# 评估输出端口。
print(f"sum: {system.GetOutputPort('sum').Eval(context)}")
print(f"difference: {system.GetOutputPort('difference').Eval(context)}")

sum: [4. 6.]
difference: [2. 2.]


这里有几点需要注意：
- 你可以声明任意数量的输入或输出（它们不需要大小相同）。
- 可以通过 `Eval()` 方法在任何接受 `Context` 的系统方法中评估输入。
- 对于每个输出端口，你需要定义不同的回调函数来实现输出。
- 端口的顺序管理可能有些脆弱。我们可以通过索引或名称引用来改进代码。

还需要理解：
- 你也可以让输入端口变为可选，通过在系统方法实现中检查 [InputPort::HasValue()](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_input_port.html#a5536b94a4642fa4cf47164437dc66ae8)。
- 输入和输出端口本身没有任何时序语义；输出端口在请求输出时被评估，并且默认[使用缓存](https://drake.mit.edu/doxygen_cxx/group__cache__design__notes.html)以避免重复计算。
- `DeclareVectorOutputPort()` 和 `DeclareAbstractOutputPort()` 可以接受一个可选的 `prerequisites_of_calc` 参数，缓存系统会用它来判断何时需要重新评估输出端口。显式缩小前置条件有时可以显著加快某些 `Diagram` 的构建速度，因为如果未指定，`DiagramBuilder` 会尝试通过符号化系统来检查代数环路。

### 2.2 抽象值端口
并非所有输入和输出都适合用向量表示。Drake 的系统框架还支持通过端口传递结构化类型，在 C++ 中通过“类型擦除”实现。从系统框架的角度看，类与“抽象”数据类型协作，只有系统实现本身需要知道如何处理结构化类型。实际上，只需忽略少量样板代码，使用起来非常直接。下面给出一个在输入和输出端口中使用 `RigidTransform` 的 `LeafSystem` 示例：

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">in &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>RotateAboutZ</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; out</td></tr></table></td></tr></table>

In [ ]:
from pydrake.math import RigidTransform, RotationMatrix

class RotateAboutZ(LeafSystem):
    def __init__(self):
        super().__init__()  # 不要忘记初始化基类。
        self.DeclareAbstractInputPort(name="in",
                                      model_value=Value(RigidTransform()))
        self.DeclareAbstractOutputPort(
            name="out",
            alloc=lambda: Value(RigidTransform()),
            calc=self.CalcOutput)

    def CalcOutput(self, context, output):
        # 评估输入端口以获得 RigidTransform。
        X_1 = system.get_input_port().Eval(context)
        # print(X_1)

        X_2 = RigidTransform(RotationMatrix.MakeZRotation(np.pi / 2)) @ X_1

        # 设置输出 RigidTransform。
        output.set_value(X_2)

# 构造该系统的实例和一个 context。
system = RotateAboutZ()
context = system.CreateDefaultContext()

# 将输入端口固定为常数值。
system.get_input_port().FixValue(context, RigidTransform())

# 评估输出端口。
print(f"output: {system.get_output_port(0).Eval(context)}")

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)
output: RigidTransform(
  R=RotationMatrix([
    [6.123233995736766e-17, -1.0, 0.0],
    [1.0, 6.123233995736766e-17, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)


## 3. 状态变量

系统将其状态存储在 `Context` 中，并以与输入和输出端口非常相似的方式处理状态。同样，我们可以拥有向量值状态或抽象值状态。抽象状态只能以离散时间方式更新，而向量值状态可以声明为**离散**或**连续**。  

离散状态 $x_d$ 会根据更新事件演化，最常见的是用简单的周期事件定义差分方程。我们在[入门笔记本](./dynamical_systems.ipynb)中已经见过一个例子。

In [ ]:
class SimpleDiscreteTimeSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        state_index = self.DeclareDiscreteState(1)  # 一个状态变量。
        self.DeclareStateOutputPort("y", state_index)  # 一个输出：y=x。
        self.DeclarePeriodicDiscreteUpdateEvent(
            period_sec=1.0,  # 一秒的时间步长。
            offset_sec=0.0,  # 第一次事件在零时刻。
            update=self.Update) # 调用下面定义的 Update 方法。

    # xk+1 = xk^3。
    def Update(self, context, discrete_state):
        x = context.get_discrete_state_vector().GetAtIndex(0)
        x_next = x**3
        discrete_state.get_mutable_vector().SetAtIndex(0, x_next)

# 实例化系统。
system = SimpleDiscreteTimeSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# 设置初始条件：x[0] = [0.9]。
context.get_mutable_discrete_state_vector().SetFromVector([0.9])

# 运行仿真。
simulator.AdvanceTo(4.0)
print(context.get_discrete_state_vector())

### 3.1 离散状态

[0.00019662705047555356]


关于这些事件的实现和顺序的详细信息可见[这里](https://drake.mit.edu/doxygen_cxx/group__discrete__systems.html)。也可以为更新离散变量定义更复杂的[事件](https://drake.mit.edu/doxygen_cxx/group__events__description.html)。

注意，系统的状态在 `Diagram` 中不会被其他系统直接访问。若要共享状态，必须通过输出端口。我们提供了 `DeclareStateOutputPort()` 方法来简化这一常见需求。

### 3.2 连续状态

连续向量值状态变量的演化由微分方程控制。虽然可以方便地声明多个离散状态组（甚至可以由不同事件以不同速率更新），但对于连续状态，我们只定义零个或一个连续状态向量，并通过重载 `LeafSystem::DoCalcTimeDerivatives()` 方法定义其动力学。下面是[入门笔记本](./dynamical_systems.ipynb)中的简单连续时间示例：

In [5]:
# 定义系统。
class SimpleContinuousTimeSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        state_index = self.DeclareContinuousState(1)  # 一个状态变量。
        self.DeclareStateOutputPort("y", state_index)  # 一个输出：y=x。

    # xdot(t) = -x(t) + x^3(t)。
    def DoCalcTimeDerivatives(self, context, derivatives):
        x = context.get_continuous_state_vector().GetAtIndex(0)
        xdot = -x + x**3
        derivatives.get_mutable_vector().SetAtIndex(0, xdot)

# 实例化系统。
system = SimpleContinuousTimeSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# 设置初始条件：x(0) = [0.9]
context.SetContinuousState([0.9])

# 运行仿真。
simulator.AdvanceTo(4.0)
print(context.get_continuous_state_vector())

[0.03778876136193669]


### 3.3 抽象状态

对于抽象值状态abstract-valued state，流程类似。在构造函数中声明抽象状态，并定义更新事件来更新该状态。离散更新事件只能更新离散状态，因此我们使用“非受限”事件来更新抽象状态。下面是一个将简单 `PiecewisePolynomial` 轨迹作为抽象状态存储的例子：

In [ ]:
class AbstractStateSystem(LeafSystem): # 抽象状态系统，存储 PiecewisePolynomial 轨迹
    # PiecewisePolynomial 是 Drake 中的一个分段多项式轨迹类，用于表示和操作随时间变化的分段多项式函数。

    def __init__(self):
        super().__init__()

        self._traj_index = self.DeclareAbstractState(
            Value(PiecewisePolynomial()))
        self.DeclarePeriodicUnrestrictedUpdateEvent(period_sec=1.0,
                                                    offset_sec=0.0,
                                                    update=self.Update)

    def Update(self, context, state):
        t = context.get_time()
        traj = PiecewisePolynomial.FirstOrderHold( # 分段多项式轨迹
            [t, t + 1],
            np.array([[-np.pi / 2.0 + 1., -np.pi / 2.0 - 1.], [-2., 2.]]))
        # 更新状态
        state.get_mutable_abstract_state(int(self._traj_index)).set_value(traj)



system = AbstractStateSystem()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# 为抽象状态设置初始条件。
context.SetAbstractState(0, PiecewisePolynomial())

# 运行仿真。
simulator.AdvanceTo(4.0)

# 仿真结束后，获取抽象状态中的轨迹，并打印其分段断点和当前时间点的轨迹值。
traj = context.get_abstract_state(0).get_value()
print(f"breaks: {traj.get_segment_times()}")
print(f"traj({context.get_time()}) = {traj.value(context.get_time())}")

breaks: [3.0, 4.0]
traj(4.0) = [[-2.57079633]
 [ 2.        ]]


## 4. 参数

参数类似于状态，但在仿真期间是常量。系统框架中的参数声明和访问方式与状态几乎相同，但参数永远不会被更新。同样，我们有向量值参数（用 `DeclareNumericParameter` 声明）和抽象值参数（用 `DeclareAbstractParameter` 声明）。

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>SystemWithParameters</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; numeric</td></tr><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; abstract</td></tr></table></td></tr></table>

In [ ]:
class SystemWithParameters(LeafSystem):
    def __init__(self):
        super().__init__()  # 不要忘记初始化基类。

        self.DeclareNumericParameter(BasicVector([1.2, 3.4]))
        self.DeclareAbstractParameter( # 抽象参数，这里以 RigidTransform 为例
            Value(RigidTransform(RotationMatrix.MakeXRotation(np.pi / 6)))) # 以绕 X 轴旋转 π/6 弧度初始化

        # 声明输出端口以演示如何在系统方法中访问参数。
        self.DeclareVectorOutputPort(name="numeric", # 数值输出端口
                                     size=2,
                                     calc=self.OutputNumeric)
        self.DeclareAbstractOutputPort( # 抽象输出端口
            name="abstract",
            alloc=lambda: Value(RigidTransform()),
            calc=self.OutputAbstract)

    def OutputNumeric(self, context, output):
        output.SetFromVector(context.get_numeric_parameter(0).get_value())

    def OutputAbstract(self, context, output):
        output.set_value(context.get_abstract_parameter(0).get_value())

# 构造该系统的实例和一个 context。
system = SystemWithParameters()
context = system.CreateDefaultContext()

# 评估输出端口。
print(f"numeric: {system.get_output_port(0).Eval(context)}")
print(f"abstract: {system.get_output_port(1).Eval(context)}")

numeric: [1.2 3.4]
abstract: RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.8660254037844387, -0.49999999999999994],
    [0.0, 0.49999999999999994, 0.8660254037844387],
  ]),
  p=[0.0, 0.0, 0.0],
)


## 5. 系统可以“发布”

除了用于更新状态的方法和用于评估输出端口的方法外，`LeafSystem` 还支持“发布”回调。发布方法不能修改任何状态：它们适用于向系统框架外部广播数据（如通过 ROS 等消息传递协议）、终止仿真、检测错误以及强制积分步之间的边界。声明方式与其他系统回调类似。

In [ ]:
class MyPublishingSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        # 调用 `ForcePublish()` 会触发回调。
        self.DeclareForcedPublishEvent(self.Publish)

        # 每秒发布一次。
        self.DeclarePeriodicPublishEvent(period_sec=1,
                                         offset_sec=0,
                                         publish=self.Publish)
        
    def Publish(self, context):
        print(f"Publish() called at time={context.get_time()}")

system = MyPublishingSystem()
simulator = Simulator(system)
simulator.AdvanceTo(5.3)

# 我们也可以在任意时刻“强制”发布。
print("\ncalling ForcedPublish:")
system.ForcedPublish(simulator.get_context())


Publish() called at time=0.0
Publish() called at time=1.0
Publish() called at time=2.0
Publish() called at time=3.0
Publish() called at time=4.0
Publish() called at time=5.0

calling ForcedPublish:
Publish() called at time=5.3


## 6. 向量值命名

抽象值端口、状态和/或参数可用于处理结构化数据。但有时用字符串名称引用向量值数据的各个元素也很方便。在 Python 中，我们推荐使用 [`namedview`](https://drake.mit.edu/pydrake/pydrake.common.containers.html?highlight=namedview#pydrake.common.containers.namedview) 工作流，其灵感来自 Python 的 [`namedtuple`](https://docs.python.org/3.6/library/collections.html?highlight=namedtuple#collections.namedtuple)，如下例所示。Drake 开发者计划[在 C++ 中提供类似功能](https://github.com/RobotLocomotion/drake/issues/12566)。

注意，连续状态向量的内存布局与离散状态、参数和输入/输出端口数据略有不同，因此处理时需要用到 `CopyToVector()` 和 `SetFromVector()`。参见 [issue #9171](https://github.com/RobotLocomotion/drake/issues/9171)。

`my_view[:]` 的用法是获取 `namedview` 本身的 NumPy 数组视图。更多细节见 [`namedview`](https://drake.mit.edu/pydrake/pydrake.common.containers.html?highlight=namedview#pydrake.common.containers.namedview) 文档中的示例和警告。

In [ ]:
# 定义系统。
class NamedViewDemo(LeafSystem):
    MyDiscreteState = namedview("MyDiscreteState", ["a", "b"]) # 离散状态
    MyContinuousState = namedview("MyContinuousState", ["x", "z", "theta"]) # 连续状态
    MyOutput = namedview("MyOutput", ["x","a"]) # 输出

    def __init__(self):
        super().__init__()

        self.DeclareDiscreteState(2) # 2 个离散状态变量。
        self.DeclarePeriodicDiscreteUpdateEvent( # 每秒更新一次离散状态
            period_sec=1.0,
            offset_sec=0.0,
            update=self.DiscreteUpdate)
        self.DeclareContinuousState(3) # 3 个连续状态变量。
        self.DeclareVectorOutputPort(name="out", size=2, calc=self.CalcOutput) # 输出端口

    def DiscreteUpdate(self, context, discrete_values): # 离散状态更新
        discrete_state = self.MyDiscreteState(context.get_discrete_state_vector().value())
        continuous_state = self.MyContinuousState(context.get_continuous_state_vector().CopyToVector())
        next_state = self.MyDiscreteState(discrete_values.get_mutable_value())
        # 现在可以通过名称引用每个元素来计算下一个状态。
        next_state.a = discrete_state.a + 1
        next_state.b = discrete_state.b + continuous_state.x

    def DoCalcTimeDerivatives(self, context, derivatives):
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        dstate_dt = self.MyContinuousState(continuous_state[:])
        dstate_dt.x = -continuous_state.x
        dstate_dt.z = -continuous_state.z
        dstate_dt.theta = -np.arctan2(continuous_state.z, continuous_state.x)
        derivatives.SetFromVector(dstate_dt[:])

    def CalcOutput(self, context, output):
        discrete_state = self.MyDiscreteState(
            context.get_discrete_state_vector().value())
        continuous_state = self.MyContinuousState(
            context.get_continuous_state_vector().CopyToVector())
        out = self.MyOutput(output.get_mutable_value())
        out.x = continuous_state.x
        out.a = discrete_state.a

# 实例化系统。
system = NamedViewDemo()
simulator = Simulator(system)
context = simulator.get_mutable_context()

# 设置初始条件。
initial_discrete_state = NamedViewDemo.MyDiscreteState([3, 4])
context.SetDiscreteState(initial_discrete_state[:])
initial_continuous_state = NamedViewDemo.MyContinuousState.Zero()
initial_continuous_state.x = 0.5
initial_continuous_state.z = 0.92
initial_continuous_state.theta = 0.23
context.SetContinuousState(initial_continuous_state[:])

# 运行仿真。
simulator.AdvanceTo(4.0)
print(
    NamedViewDemo.MyDiscreteState(context.get_discrete_state_vector().value()))
print(
    NamedViewDemo.MyContinuousState(
        context.get_continuous_state_vector().CopyToVector()))
print(NamedViewDemo.MyOutput(system.get_output_port().Eval(context)))


MyDiscreteState(a=7.0, b=4.776488676855167)
MyContinuousState(x=0.009156362264504603, z=0.016847706566688464, theta=8.504474637647576)
MyOutput(x=0.009156362264504603, a=7.0)


## 7. 支持标量类型转换（double、AutoDiff 和 Symbolic）

为了在系统框架中支持 AutoDiff 和 Symbolic 类型，LeafSystem 可以编写为支持“标量类型转换”。在 Python 中，只需少量样板代码即可实现。下面是一个简单示例：

<table align=center cellpadding=0 cellspacing=0><tr align=center><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">state &rarr;</td></tr><tr><td align=right style="padding:5px 0px 5px 0px; border:none;">command &rarr;</td></tr></table></td><td align=center style="border:2px solid black;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor=#F0F0F0>RunningCost</td><td style="vertical-align:middle; border:none;"><table cellspacing=0 cellpadding=0><tr><td align=left style="padding:5px 0px 5px 0px; border:none;">&rarr; cost</td></tr></table></td></tr></table>

In [ ]:
from pydrake.systems.framework import LeafSystem_
from pydrake.systems.scalar_conversion import TemplateSystem
from pydrake.autodiffutils import AutoDiffXd
from pydrake.symbolic import Expression

@TemplateSystem.define("RunningCost_")  #  Drake 的模板系统装饰器，使得 RunningCost_ 可以根据不同的标量类型（T）生成不同类型的系统（如 float、自动微分、符号运算）。
def RunningCost_(T):

    class Impl(LeafSystem_[T]):

        def _construct(self, converter=None, Q=np.eye(2)):
            super().__init__(converter)
            self._Q = Q
            self._state_port = self.DeclareVectorInputPort("state", 2)
            self._command_port = self.DeclareVectorInputPort("command", 1)
            self.DeclareVectorOutputPort("cost", 1, self.CostOutput)
 
        def _construct_copy(self, other, converter=None): # 用于标量类型转换时复制成员变量。
            # 任何成员字段（如配置值）都需要
            # 在这里从 `other` 传递到 `self`。
            Impl._construct(self, converter=converter, Q=other._Q)

        def CostOutput(self, context, output): # 计算运行成本
            x = self._state_port.Eval(context)
            u = self._command_port.Eval(context)[0]
            output[0] = x.dot(self._Q.dot(x)) + u**2

    return Impl

RunningCost = RunningCost_[None]  # 默认实例化。

关键步骤如下：
- 添加 `@TemplateSystem` 装饰器。
- 继承自 `LeafSystem_[T]` 而不是简单的 `LeafSystem`。
- 实现 `_construct` 方法，*代替*通常的 `__init__` 方法。
- 实现 `_construct_copy` 方法，需要将与 `_construct` 相同的成员字段（如本例中的 `self.Q`）赋值。
- 添加默认实例化，这样你可以像原来一样用 `RunningCost`，也可以用 `RunningCost_[float]`。

更多细节可见 C++ 的[标量转换文档](https://drake.mit.edu/doxygen_cxx/group__system__scalar__conversion.html)和 [`@TemplateSystem`](https://drake.mit.edu/pydrake/pydrake.systems.scalar_conversion.html#pydrake.systems.scalar_conversion.TemplateSystem) 装饰器文档。

In [ ]:
# 完成上述操作后，仍可像原来一样使用该系统：
# 创建一个二次型权重为 diag([10, 1]) 的 RunningCost 系统。
# 创建 context 并设置输入端口（state=[1,2]，command=[3]）。
#  计算并打印代价输出（cost = xᵀQx + u²）。
system = RunningCost(Q=np.diag([10, 1]))
context = system.CreateDefaultContext()
system.get_input_port(0).FixValue(context, [1, 2])
system.get_input_port(1).FixValue(context, [3])
print(system.get_output_port().Eval(context))

# 也可以直接声明 autodiff 或符号类型的系统：
# 直接实例化支持自动微分（AutoDiffXd）和符号运算（Expression）的 RunningCost 系统，便于做灵敏度分析或符号推导。
system_ad = RunningCost_[AutoDiffXd]()
system_symbolic = RunningCost_[Expression]()

# 或通过标量转换：
# 通过已有系统对象直接转换为自动微分或符号类型的系统，方便复用已有配置。
system_ad = system.ToAutoDiffXd()
system_symbolic = system.ToSymbolic()

# 还可以将时间、状态、参数以及（如有需要）输入端口
# 从原系统转换到新系统：
# 创建符号系统的 context，并把原 context 的时间、状态、参数、输入端口全部同步过去。
# 这样可以在不同类型系统间无缝切换，最后计算并打印符号系统下的输出。
context_symbolic = system_symbolic.CreateDefaultContext()
context_symbolic.SetTimeStateAndParametersFrom(context)
system_symbolic.FixInputPortsFrom(system, context, context_symbolic)
print(system_symbolic.get_output_port().Eval(context_symbolic))

[23.]
[<Expression "23">]


## 8. 延伸阅读

[系统框架中的缓存](https://drake.mit.edu/doxygen_cxx/group__cache__design__notes.html)

[声明事件](https://drake.mit.edu/doxygen_cxx/group__events__description.html)

[随机系统](https://drake.mit.edu/doxygen_cxx/group__stochastic__systems.html)

声明[系统约束](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system_constraint.html)：[等式约束](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html#a4948ad0241c67045b3c794874b2986a0)与[不等式约束](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html#a3bac306621c3f0839324649151c22af2)。

[用隐式形式编写连续动力学](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system.html#a2bb4c1e3572a8009863b5a342fcb5c49)